## Clustering de Clientes por Consumo de Energia - Modelagem

#### Imports

In [1]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

#### Dataset

In [6]:
df = pd.read_csv('..\data\household_power_consumption.txt', sep=',', low_memory=False)
df.drop(['Date','Time'], axis=1, inplace=True)
df

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,4.216,0.418,234.84,18.4,0.0,1.0,17.0
1,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2,5.374,0.498,233.29,23.0,0.0,2.0,17.0
3,5.388,0.502,233.74,23.0,0.0,1.0,17.0
4,3.666,0.528,235.68,15.8,0.0,1.0,17.0
...,...,...,...,...,...,...,...
2049275,0.946,0.000,240.43,4.0,0.0,0.0,0.0
2049276,0.944,0.000,240.00,4.0,0.0,0.0,0.0
2049277,0.938,0.000,239.82,3.8,0.0,0.0,0.0
2049278,0.934,0.000,239.70,3.8,0.0,0.0,0.0


#### Modelagem

In [7]:
# Aplicando redução de dimensionalidade
df = df.values
pca = PCA(n_components=2).fit_transform(df)

In [8]:
# Procurando o melhor valor de K
k_range = range(1,12)
k_means_var = [KMeans(n_clusters=k).fit(pca) for k in k_range]

In [9]:
# Centróides de cada modelo
centroids = [x.cluster_centers_ for x in k_means_var]